In [1]:
import os
file_path = os.path.join(".","train_data","xiaohuangji50w_nofenci.conv")
convs = []
with open(file_path, encoding="utf-8") as f:
    one_conv = []
    for line in f:
        line = line.strip("\n").replace("/","")
        if line == "":
            continue
        if line[0] == "M":
            one_conv.append(line.split(" ")[1])
        elif line[0] == "E":
            if one_conv:
                convs.append(one_conv)
            one_conv = []

In [11]:
print(len(convs))
print([conv for conv in convs[13:16]])

454130
[['许兵到底是谁', '不要凶我～应该大概也许是叶祺吧～'], ['尼玛,许兵到底是谁', '我。。。我不会告诉你我是小澜的！(^V^)'], ['小黄鸭,你有女朋友么', '老娘是女的…！']]


In [12]:
import jieba 
seq = []
sep_token = "\t"
for conv in convs:
    if len(conv) == 1:
        continue
    if len(conv)%2 != 0:
        conv = conv[:-1]
    for i in range(len(conv)):
        if i%2 ==0:
            conv[i] = " ".join(jieba.cut(conv[i]))
            conv[i+1] = " ".join(jieba.cut(conv[i+1]))
            seq.append(conv[i] + sep_token + conv[i+1])

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\lucif\AppData\Local\Temp\jieba.cache
Loading model cost 0.724 seconds.
Prefix dict has been built succesfully.


In [16]:
print(len(seq))
print([conv for conv in seq[13:16]])

454130
['许兵 到底 是 谁\t不要 凶 我 ～ 应该 大概 也许 是 叶祺 吧 ～', '尼玛 , 许兵 到底 是 谁\t我 。 。 。 我 不会 告诉 你 我 是 小澜 的 ！ ( ^ V ^ )', '小黄 鸭 , 你 有 女朋友 么\t老娘 是 女 的 … ！']


In [17]:
seqdata_path = os.path.join(".","train_data","seq.data")
if not os.path.isfile(seqdata_path):
    content = ""
    for i in range(len(seq)):
        content += seq[i] + "\n"
        if i%1000 == 0:
            print(".",end="")
    with open(seqdata_path, "w", encoding="utf-8") as f:
        f.write(content)
        print("finish preprocessing seq data")
    del content
else:
    print("seq.data Exist")
    
del convs, seq

seq.data Exist


In [28]:
import io

num_examples = 130000 # 必須小於454130
start_token = "start"
end_token = "end"
sep_token = "\t"
def preprocess_sentence(w):
    w = start_token+" "+w+" "+end_token
    return w

seqdata_path = os.path.join(".","train_data","seq.data")
lines = io.open(seqdata_path, encoding="utf-8").read().strip().split("\n")
word_pairs = [[preprocess_sentence(w) for w in l.split(sep_token)] for l in lines[:num_examples]]
input_lang, target_lang = zip(*word_pairs)

del lines, word_pairs

In [76]:
print([input_lang[13:16]])
print([target_lang[13:16]])

[('start 许兵 到底 是 谁 end', 'start 尼玛 , 许兵 到底 是 谁 end', 'start 小黄 鸭 , 你 有 女朋友 么 end')]
[('start 不要 凶 我 ～ 应该 大概 也许 是 叶祺 吧 ～ end', 'start 我 。 。 。 我 不会 告诉 你 我 是 小澜 的 ！ ( ^ V ^ ) end', 'start 老娘 是 女 的 … ！ end')]


In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 20000

def tokenize(lang):
    lang_tokenizer = Tokenizer(num_words=vocab_size, oov_token=3)
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang) # 把 文字 轉為 index
    tensor = pad_sequences(tensor, padding="post") # 長度不足的句子補空白
    return tensor, lang_tokenizer

input_tensor, input_tokenizer = tokenize(input_lang)
target_tensor, target_tokenizer = tokenize(target_lang)

In [88]:
print("vocab_size",input_tokenizer.num_words, target_tokenizer.num_words)
print("input_tensor.shape",input_tensor.shape)
print("target_tensor.shape",target_tensor.shape)
print(input_tensor[13][0:13])
print([input_tokenizer.index_word.get(index) for index in input_tensor[13][0:13]])
print(target_tensor[13][0:13])
print([target_tokenizer.index_word.get(index) for index in target_tensor[13][0:13]])

max_length_input = max(len(t) for t in input_tensor)
max_length_target = max(len(t) for t in target_tensor)
print("問句最長",max_length_input)
print("答句最長",max_length_target)

vocab_size 20000 20000
input_tensor.shape (130000, 135)
target_tensor.shape (130000, 173)
[   2 4003  174    7   13    3    0    0    0    0    0    0    0]
['start', '许兵', '到底', '是', '谁', 'end', None, None, None, None, None, None, None]
[    3    36  2684     5    22   244   613   988    11 13284    20    22
     2]
['start', '不要', '凶', '我', '～', '应该', '大概', '也许', '是', '叶祺', '吧', '～', 'end']
問句最長 135
答句最長 173


In [115]:
from tensorflow.keras import layers, models
from tensorflow import keras
embedding_op_dim = 32
gru_op_dim = 256

model = models.Sequential()
model.add(keras.Input(shape=(135,), name="inputs"))
model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_op_dim, name="embed_input"))
model.add(layers.GRU(units=gru_op_dim, recurrent_initializer="glorot_uniform", name="gru_input"))# , return_sequences=True,return_state=True
model.add(layers.Flatten())
model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_op_dim, name="embed_target"))
model.add(layers.GRU(units=gru_op_dim, recurrent_initializer="glorot_uniform", name="gru_target"))# , return_sequences=True,return_state=True
model.add(layers.Dense(units=173, name="outputs", activation="softmax"))
model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed_input (Embedding)      (None, 135, 32)           640000    
_________________________________________________________________
gru_input (GRU)              (None, 256)               222720    
_________________________________________________________________
flatten_9 (Flatten)          (None, 256)               0         
_________________________________________________________________
embed_target (Embedding)     (None, 256, 32)           640000    
_________________________________________________________________
gru_target (GRU)             (None, 256)               222720    
_________________________________________________________________
outputs (Dense)              (None, 173)               44461     
Total params: 1,769,901
Trainable params: 1,769,901
Non-trainable params: 0
____________________________________________

In [111]:
optimizer = keras.optimizers.Adam(lr=0.05)
loss = keras.losses.SparseCategoricalCrossentropy() 
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [113]:
history = model.fit(input_tensor, target_tensor, batch_size=128, epochs=1, validation_split=0.05)

Train on 123500 samples, validate on 6500 samples
  1024/123500 [..............................] - ETA: 1:10:02 - loss: 85307725.1429

KeyboardInterrupt: 